# Transformer-based Language Model in Pytorch

Remarque : fait à l'arrache dans le notebook pour l'instant, déplacement dans un .py plus tard

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
import sys
import torchtext
sys.path.append("..")
#from common import get_positional_encodings
import nltk

In [2]:
#entrée : positional encoding + embeddings
#ajouter layer norm

In [3]:
# Hyperparamètres

nb_decoders = 6
nb_heads = 8
vector_size = 512
vocab_size = 1000
head_size = vector_size//nb_heads
ffn_hidden_size = 100
seq_length = 15
attention_features_size = 5


In [4]:
class Transformer(nn.Module):
    "Whole transformer structure composed of stacked decoder blocks"
    def __init__(self, decoder):
        super(Transformer, self).__init__()
        self.decoder = decoder
        self.embedding = nn.Embedding(vocab_size, 4)
        self.finalfc = nn.Linear(vector_size, vocab_size)

    def forward(self, x):
        
        embedded = embedding(x)
        pos_encodings = torch.tensor(get_positional_encodings(seq_length, vector_size))
        x = embedded + pos_encodings
        
        for x in range(nb_decoders):
            x = self.decoder(x)
            
        x = F.softmax(self.finalfc(x))
        return x

In [5]:
class Decoder(nn.Module):
    "Decoder block applying multihead attention mechanism followed by a feedforward network"
    def __init__(self, multihead_attention, feedforward_network):
        super(Decoder, self).__init__()
        self.multihead_attention = multihead_attention
        self.feedforward_network = feedforward_network
        self.layernorm = nn.modules.normalization.LayerNorm

    def forward(self, x):
        #normalisation a la fin ou au debut ?
        mha = self.multihead_attention(self.layernorm(x))
        x += mha
        ffo = self.feedforward_network(self.layernorm(x))
        x += ffo
        return x
    

In [6]:
class FeedforwardNetwork(nn.Module):
    "Classic Feedforward Network with two layers"
    def __init__(self, vector_size, hidden_size):
        super(FeedforwardNetwork, self).__init__()
        self.hidden_size  = hidden_size
        self.fc1 = nn.Linear(vector_size, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, vector_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
ffn = FeedforwardNetwork(10,5)

Tests pour compréhension opérations torch, à virer

In [79]:
a = torch.tensor([[1,2,3],[3,4,5]])
b = torch.tensor([[1,2,1],[1,1,1],[3,3,3]])

In [84]:
a = torch.tensor([[[[1,2],[3,4]]],[[[1,2],[3,4]]]])
b = torch.tensor([[[[1,1],[10,20]],[[1,1],[10,20]]],[[[1,1],[10,20]],[[1,1],[10,20]]]])

In [117]:
b.transpose(-1,-2)

tensor([[[[ 1, 10],
          [ 1, 20]],

         [[ 1, 10],
          [ 1, 20]]],


        [[[ 1, 10],
          [ 1, 20]],

         [[ 1, 10],
          [ 1, 20]]]])

In [86]:
torch.matmul(a,b)

tensor([[[[21, 41],
          [43, 83]],

         [[21, 41],
          [43, 83]]],


        [[[21, 41],
          [43, 83]],

         [[21, 41],
          [43, 83]]]])

In [99]:
b.float()

tensor([[[[ 1.,  1.],
          [10., 20.]],

         [[ 1.,  1.],
          [10., 20.]]],


        [[[ 1.,  1.],
          [10., 20.]],

         [[ 1.,  1.],
          [10., 20.]]]])

In [118]:
torch.softmax(b.float(), dim=-1)

tensor([[[[5.0000e-01, 5.0000e-01],
          [4.5398e-05, 9.9995e-01]],

         [[5.0000e-01, 5.0000e-01],
          [4.5398e-05, 9.9995e-01]]],


        [[[5.0000e-01, 5.0000e-01],
          [4.5398e-05, 9.9995e-01]],

         [[5.0000e-01, 5.0000e-01],
          [4.5398e-05, 9.9995e-01]]]])

In [107]:
b.resize(2*2*2,2)

tensor([[ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20]])

In [115]:
b.view(-1,2)

tensor([[ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20],
        [ 1,  1],
        [10, 20]])

In [116]:
b.shape[-1]

2

In [123]:
torch.randn(5,3)

tensor([[-0.6129, -1.3946, -1.7488],
        [ 1.0109,  1.1781,  1.2424],
        [-1.1034,  2.5773, -0.4679],
        [ 0.4369, -0.4911, -0.2773],
        [ 1.0198,  0.2498, -2.3137]])

In [130]:
torch.randn(20).view(2,2,5)

tensor([[[-0.6981, -1.8573, -0.2192, -0.8049,  0.4418],
         [-0.0916, -0.2362, -0.5362,  0.8284, -0.7785]],

        [[ 1.4417,  3.0020,  0.0091,  1.0252, -0.0826],
         [-0.5025, -1.5500, -1.4086, -0.5481,  0.6857]]])

In [194]:
lineartest = nn.Linear(6,8)
a = lineartest(torch.randn(2,6))

In [195]:
a

tensor([[-0.0635,  0.0135, -0.1823, -0.6612, -0.0111,  0.1565, -0.4600,  0.1574],
        [-0.0561,  0.7180,  0.8235,  0.3773, -0.6609, -0.1302, -0.1962, -0.2220]],
       grad_fn=<AddmmBackward>)

In [151]:
a.reshape(5,2,4).transpose(1,0)

tensor([[[ 0.1325,  0.8591,  0.0375,  0.3665],
         [-0.0203,  0.5444,  0.4884,  0.3979],
         [ 0.2096,  0.4511,  0.4392, -0.1969],
         [-0.1197,  0.7166,  0.3569,  0.8327],
         [ 0.0605,  0.4559,  0.5451,  0.0421]],

        [[-0.4523,  0.3027, -0.2264,  0.4526],
         [-1.0083, -0.0689,  0.8458, -0.8348],
         [-1.2110,  0.1211, -0.0691,  0.1079],
         [-0.6316, -0.0602,  1.1216, -1.0492],
         [-1.2694, -0.0477,  0.5144, -0.5315]]], grad_fn=<TransposeBackward0>)

In [47]:
a = torch.randn(2,5,4)

In [58]:
a.repeat(2, 1, 1, 1)

tensor([[[[-0.0612,  1.2455, -0.4738, -1.0857],
          [ 0.0181,  1.2753,  0.6034,  0.1763],
          [ 0.3451,  0.9108,  1.3092, -0.7899],
          [-0.3710,  0.9705, -1.3661,  1.0217],
          [ 1.3385, -0.3061, -0.5540,  1.0602]],

         [[-0.9749,  0.6760, -0.6670,  0.7981],
          [-0.6257, -0.6794, -1.1734,  0.0268],
          [-1.5674, -0.1366,  0.7900,  0.0707],
          [ 0.6286, -2.2059,  0.5805,  0.2560],
          [ 1.1570,  0.5268,  0.0600,  0.5756]]],


        [[[-0.0612,  1.2455, -0.4738, -1.0857],
          [ 0.0181,  1.2753,  0.6034,  0.1763],
          [ 0.3451,  0.9108,  1.3092, -0.7899],
          [-0.3710,  0.9705, -1.3661,  1.0217],
          [ 1.3385, -0.3061, -0.5540,  1.0602]],

         [[-0.9749,  0.6760, -0.6670,  0.7981],
          [-0.6257, -0.6794, -1.1734,  0.0268],
          [-1.5674, -0.1366,  0.7900,  0.0707],
          [ 0.6286, -2.2059,  0.5805,  0.2560],
          [ 1.1570,  0.5268,  0.0600,  0.5756]]]])

In [62]:
a.transpose(-2,-3).reshape(5,8)

tensor([[-0.0612,  1.2455, -0.4738, -1.0857, -0.9749,  0.6760, -0.6670,  0.7981],
        [ 0.0181,  1.2753,  0.6034,  0.1763, -0.6257, -0.6794, -1.1734,  0.0268],
        [ 0.3451,  0.9108,  1.3092, -0.7899, -1.5674, -0.1366,  0.7900,  0.0707],
        [-0.3710,  0.9705, -1.3661,  1.0217,  0.6286, -2.2059,  0.5805,  0.2560],
        [ 1.3385, -0.3061, -0.5540,  1.0602,  1.1570,  0.5268,  0.0600,  0.5756]])

In [93]:
#Mask ?

class MultiHeadAttention(nn.Module):
    "MultiHead Attention Block"
    def __init__(self, nb_heads, head_size, vector_size):
        super(MultiHeadAttention, self).__init__()
        self.nb_heads = nb_heads
        self.head_size = head_size
        self.vector_size = vector_size
        #bias = False ? (pour équivalence stricte avec une multiplication matricielle)
        self.w_q = nn.Linear(vector_size, vector_size)
        self.w_k = nn.Linear(vector_size, vector_size)
        self.w_v = nn.Linear(vector_size, vector_size)
        self.w_0 = nn.Linear(vector_size, vector_size)
        
    def mask_w(w):
        #
        return w
    
    def reshape_w(self, w):
        #reshape a matrix (batch_size, nb_inputs, vector_size)
        #towards a matrix (batch_size, nb_heads, nb_inputs, head_size)
        w = w.reshape(-1, w.shape[1], self.nb_heads, self.head_size)
        w = w.transpose(-2,-3)
        return w
        
    def forward(self, x):
        
        # x size (batch_size, nb_inputs, vector_size)
        # self.w_q(x) size (batch_size, nb_inputs, vector_size)
        q = self.reshape_w(self.w_q(x))
        # q size (batch_size, nb_heads, nb_inputs, head_size)
        k = self.reshape_w(self.w_k(x))
        # k size (batch_size, nb_heads, nb_inputs, head_size)
        v = self.reshape_w(self.w_v(x))
        # v size (batch_size, nb_heads, nb_inputs, head_size)
        
        w = torch.matmul(q, k.transpose(-1,-2)) / math.sqrt(k.shape[-1])
        w = torch.softmax(w, dim=-1)
        # w size (batch_size, nb_heads, nb_inputs, nb_inputs)
        
        a = torch.matmul(w, v)
        # a size (batch_size, nb_heads, nb_inputs, head_size)
        
        a = a.transpose(-2,-3).reshape(-1, a.shape[-2], self.vector_size)
        # a size (batch_size, nb_inputs, vector_size) (concatenation of multihead matrices)
        
        a = self.w_0(a)
        # a size (batch_size, nb_inputs, vector_size)

        return a


In [94]:
#Test unitaire

In [95]:
atest = torch.randn(5,8)
mha = MultiHeadAttention(2,4,8)

In [99]:
mha(atest.repeat(3,1,1))

torch.Size([3, 5, 8])
torch.Size([3, 5, 8])


tensor([[[ 0.2979, -0.1799,  0.0621, -0.0203,  0.0888,  0.4438,  0.0339,
           0.1888],
         [ 0.2768, -0.2103,  0.0576, -0.0086,  0.0801,  0.4732,  0.0374,
           0.2023],
         [ 0.2855, -0.2163,  0.0293, -0.0142,  0.1087,  0.4576,  0.0604,
           0.1691],
         [ 0.2821, -0.1908,  0.0428,  0.0080,  0.0760,  0.3977,  0.0773,
           0.1864],
         [ 0.2620, -0.2677,  0.0191,  0.0256,  0.0799,  0.4859,  0.0352,
           0.1899]],

        [[ 0.2979, -0.1799,  0.0621, -0.0203,  0.0888,  0.4438,  0.0339,
           0.1888],
         [ 0.2768, -0.2103,  0.0576, -0.0086,  0.0801,  0.4732,  0.0374,
           0.2023],
         [ 0.2855, -0.2163,  0.0293, -0.0142,  0.1087,  0.4576,  0.0604,
           0.1691],
         [ 0.2821, -0.1908,  0.0428,  0.0080,  0.0760,  0.3977,  0.0773,
           0.1864],
         [ 0.2620, -0.2677,  0.0191,  0.0256,  0.0799,  0.4859,  0.0352,
           0.1899]],

        [[ 0.2979, -0.1799,  0.0621, -0.0203,  0.0888,  0.4438,  0

In [8]:
with open("fr.train.top1M.txt","r",encoding="utf8") as file:
    text = file.read()

a


In [ ]:
tokens = nltk.word_tokenize(text)

In [27]:
LMtransformer = Transformer(Decoder(MultiHeadAttention(nb_heads, head_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

criterion = nn.CrossEntropyLoss() #perplexity ?
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.001)

In [18]:
def train_model(nb_epochs, batch_size):
    LMtransformer.train()
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        for i in range(len(tokens)//batch_size):
            
            batch, y = tokens[i: i+batch_size], tokens[i+batch_size]
            optimizer.zero_grad()
            output = LMtransformer(batch)
            loss = criterion(output, y)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.